### Imports and definitions

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import struct

In [ ]:
class Color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

### Fibonacci Sequence

In [ ]:
N = 8

In [ ]:
# Print the first N Fibonacci numbers
fib_0 = 0
fib_1 = 1
print(str(fib_0) + '\n' + str((fib_1)))
for i in range(N):
    tmp = fib_1
    fib_1 = fib_0 + fib_1
    fib_0 = tmp
    print(fib_1)

In [ ]:
# Generate the list of the first N Fibonacci numbers
fib = [0, 1]
for i in range(N):
    fib.append(fib[-1] + fib[-2])

print(fib)

### Integer Fibonacci coding

In [ ]:
N = 5000

values = []
codewords = []

last_valid_value = 0
last_valid_bits = ['0']

for i in range(N):
    
    bits = list(bin(i))[2:]
    count = 0
    keep = True
    for j in range(len(bits)):
        if bits[j] == '1':
            count += 1
            if count >= 2:
                keep = False
                break
        else:
            count = 0
    if keep:
        last_valid_value = i
        last_valid_bits = bits
        
    values.append(last_valid_value) # If a value is not fibonacci-valid we use the last fibonacci-valid value instead
    codewords.append(''.join(last_valid_bits))

In [ ]:
fig = plt.figure(figsize=(20,10))
fig.suptitle("Fibonacci codings of the " + repr(N) + " first integers (rounded down)", fontsize=20)
plt.plot(list(range(N)), values)
plt.show()

In [ ]:
N = 5000

values = []
codewords = []

last_valid_value = 0
last_valid_bits = ['0']
count_not_keep = 0

for i in range(N):
    
    bits = list(bin(i))[2:]
    count = 0
    keep = True
    for j in range(len(bits)):
        if bits[j] == '1':
            count += 1
            if count >= 2:
                keep = False
                break
        else:
            count = 0
    if keep:
        last_valid_value = i
        last_valid_bits = bits
        # Change the previous fibonacci-valid values so that they are closer to their actual value
        for j in range(i - (math.floor(count_not_keep/2)), i):
            values[j] = last_valid_value
            codewords[j] = ''.join(last_valid_bits)
        count_not_keep = 0
        
    else:
        count_not_keep += 1
        
    values.append(last_valid_value)
    codewords.append(''.join(last_valid_bits))

In [ ]:
fig = plt.figure(figsize=(20,10))
fig.suptitle("Fibonacci codings of the " + repr(N) + " first integers (rounded to the closest)", fontsize=20)
plt.plot(list(range(N)), values)
plt.show()

### Floating-point (numpy.float32) Fibonacci coding

In [ ]:
print("Full precision value = %1.52f" % 2.432);
print("Full precision value = %1.52f" % np.float32(2.432))

As we can see python has 64-bits precision for float type numbers. In this section we will use np.float32 numbers instead.

In [ ]:
# Gives the 32-bit floating point representation of a number in native python float type (64-bits)
def f32(num):
    return np.float32(num)

In [ ]:
# Gives the binary value of a floating point number as a string
def binary(num):
    return ''.join(bin(c).replace('0b', '').rjust(8, '0') for c in struct.pack('!f', num))

In [ ]:
# Tests of the binary function, the binary value on the right was obtained using an online float-to-binary converter
assert(binary(f32(0.)) == '00000000000000000000000000000000')
assert(binary(f32(2.432)) == '01000000000110111010010111100011')
assert(binary(f32(-2.432)) == '11000000000110111010010111100011')
assert(binary(f32(-3.4028235E38)) == '11111111011111111111111111111111')
assert(binary(f32(3.4028235E38)) == '01111111011111111111111111111111')
assert(binary(f32(1.4E-45)) == '00000000000000000000000000000001')
assert(binary(f32(-0.0)) == '10000000000000000000000000000000')
print("Tests passed!")

In [ ]:
# Gives the floating point number from its binary string
def float_from_bin(code):
    return f32(struct.unpack('f', struct.pack('I', int(code, 2)))[0])

In [ ]:
# Tests of the float_from_bin function
assert(float_from_bin('00000000000000000000000000000000') == f32(0.0))
assert(float_from_bin('01000000000110111010010111100011') == f32(2.432))
assert(float_from_bin('11000000000110111010010111100011') == f32(-2.432))
assert(float_from_bin('11111111011111111111111111111111') == f32(-3.4028235E38))
assert(float_from_bin('01111111011111111111111111111111') == f32(3.4028235E38))
assert(float_from_bin('00000000000000000000000000000001') == f32(1.4E-45))
assert(float_from_bin('10000000000000000000000000000000') == f32(-0.0))
print("Tests passed!")

In [ ]:
# This version only rounds down, need to check if rounding up is better
# Rounding up will give other extreme cases to take care of:
# The exponent part could overflow to the sign part, the mantissa could overflow to the exponent part, the sign could overflow

# Gives the Fibonacci-valid float-32 number that is the closest to our float-32 number 'num'
def fib_code(num):
    code = list(binary(num))
    count = 0
    for i in range(len(code)):
        if code[i] == '1':
            count += 1
            if count >= 2:
                code[i] = '0' # Remove the problem
                # Then we need to make the number as big as possible by placing only 10101010101... until the end of the floating point representation
                one_next = True
                for j in range(i+1, len(code)):
                    if one_next:
                        code[j] = '1'
                        one_next = False
                    else:
                        code[j] = '0'
                        one_next = True
                break
        else:
            count = 0

    code = ''.join(code)
    return float_from_bin(code)

### Testing and current problems

In [ ]:
# Test with a few values
numbers = [2.432, 1239481231., 1230412.3210312, 312319., math.pow(2, 17), math.pow(2, 16), 123456.123456, 123.123, 987.987,
          2.2737368E-13, 1.0, 0.0, -2.432, -0.0, -2131231.0, -0.000012391, -1.6434601E-32]
column = 20

print(Color.BLUE + Color.BOLD + "Number".ljust(column) + "Fib down\n".ljust(column) + Color.END)

for n in numbers:
    print((repr(n)).ljust(column) + repr(fib_code(n)).ljust(column))

In [ ]:
abs(math.pow(2, 32) - math.pow(2,33))

In [ ]:
abs(math.pow(2, 32) - 5592405)

So the Fibonacci-valid number to 2^32 is 5592405 and not 2^33

Also the smallest Fibonacci-valid number is -3.0316488E-13 (which is very close to zero).
This is because we cannot get a positive exponent for negative numbers (because of the leading one).

### Integer (numpy.int32) Fibonacci coding

In [ ]:
def binary_int(num, bits=32):
    code = bin(num)[2:]
    if len(code) > bits:
        print("ERROR: num needs more bits")
        return '0' * bits
    else:
        return '0' * (bits-len(code)) + code

In [ ]:
def int_from_bin(code):
    return int(code, 2)

In [ ]:
# Gives the Fibonacci-valid int number that is the closest (down) to our int number 'num'
def fib_code_int_down(num, bits=32):
    code = list(binary_int(num, bits))
    count = 0
    for i in range(len(code)):
        if code[i] == '1':
            count += 1
            if count >= 2:
                code[i] = '0' # Remove the problem
                # Then we need to make the number as big as possible by placing only 10101010101... until the end of the floating point representation
                one_next = True
                for j in range(i+1, len(code)):
                    if one_next:
                        code[j] = '1'
                        one_next = False
                    else:
                        code[j] = '0'
                        one_next = True
                break
        else:
            count = 0

    code = ''.join(code)
    return int_from_bin(code)

In [ ]:
# Gives the Fibonacci-valid int number that is the closest (up) to our int number 'num'
def fib_code_int_up(num, bits=32):
    code = list(binary_int(num, bits))
    count = 0
    for i in range(len(code)):
        if code[i] == '1':
            count += 1
            if count >= 2:

                one_next = True
                for j in range(i-2, -1, -1):
                    if one_next:
                        code[j] = '1'
                        one_next = False
                    else:
                        code[j] = '0'
                        one_next = True
                    if j > 0 and code[j-1] == '0':
                        break;
                
                i -= 1
                while i < len(code):
                    code[i] = '0' # Remove the problem (and all subsequent problems on the right)
                    i += 1
                break
                                
        else:
            count = 0

    code = ''.join(code)
    return int_from_bin(code)

In [ ]:
# Gives the best Fibonacci-valid int approximation for 'num'
def fib_code_int(num, bits=32):
    down = fib_code_int_down(num, bits)
    up = fib_code_int_up(num, bits)
    dist_down = abs(num - down)
    dist_up = abs(num - up)
    if dist_down < dist_up:
        return down
    else:
        return up

### Testing and problems

In [ ]:
# 32 bits
numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100, 1000, 2731, 4000, 4095, 4096, 123456]
column = 10

print(Color.BLUE + Color.BOLD + "Number".ljust(column) + "Fib down".ljust(column) + "Fib up".ljust(column) + "Best Fib\n".ljust(column) + Color.END)

for n in numbers:
    print((repr(n)).ljust(column) + repr(fib_code_int_down(n)).ljust(column) + repr(fib_code_int_up(n)).ljust(column) + repr(fib_code_int(n)).ljust(column))

It does not work yet with negative numbers. Negative numbers would get very very bad Fibonnaci quantizations given how the 2s complement works in binary.

In [ ]:
# 8 bits
numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100, 125, 180, 240, 255]
column = 10
bits = 8

print(Color.BLUE + Color.BOLD + "Number".ljust(column) + "Fib down".ljust(column) + "Fib up".ljust(column) + "Best Fib\n".ljust(column) + Color.END)

for n in numbers:
    print((repr(n)).ljust(column) + repr(fib_code_int_down(n, bits=bits)).ljust(column) + repr(fib_code_int_up(n, bits=bits)).ljust(column) + repr(fib_code_int(n, bits=bits)).ljust(column))